In [19]:
%load_ext autoreload
%autoreload 2

from core.optimizer_evaluator import *
from core.analzyer import *
from tabulate import tabulate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Тесты для SGD с разным размером батча

В тестах генерируются случайные квадратичные формы и оценивается количество итераций алгоритма до достижения минимума с заданной точностью.

Для тестов был выбран именно подобный класс функций, так как он позволяет генерировать формы с заданным числом обусловленности, что позволяет регулировать сложность функции. Помимо этого генерируется такая форма, что минимум всегда находится в точке $0_n$ и равен 0, что упрощяет оценку качества сходимости разных методов.

In [20]:
def visualize_batch_testing(maxbatch, scheduler):
    tests = 10
    data = { 'test': [] }

    x0 = np.array([-5, 5])

    for i in range(1, tests + 1):
        fs = [generate_positive_definite_quadratic_form(2, 10, random_orthonormal_basis) for _ in range(maxbatch)]
        dfs = [f.gradient_function() for f in fs]
        result = test_batch(fs, dfs, x0, scheduler)
        data['test'].append(i)

        for key, value in result:
            if key not in data:
                data[key] = []
            data[key].append(value)


    print(tabulate(data, headers="keys", tablefmt="grid"))

### Подбор фукнции изменения шага

Фиксированный шаг дает неплохие результаты по сравнению с другими методами

In [21]:
visualize_batch_testing(5, lambda batch: fixed_step_search(1))

+--------+-------------+-------------+-------------+-------------+-------------+
|   test |   batch = 1 |   batch = 2 |   batch = 3 |   batch = 4 |   batch = 5 |
+========+=============+=============+=============+=============+=============+
|      1 |          16 |          13 |          12 |          11 |          10 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      2 |          16 |          13 |          11 |          11 |          10 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      3 |          16 |          13 |          12 |          11 |          10 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      4 |          17 |          13 |          12 |          11 |          10 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      5 |          16 |          13 |          12 |          11 |          10 |
+--------+-------------+----

In [22]:
visualize_batch_testing(5, lambda batch: step_learning_scheduler(10, 1, 1, batch, 5))

+--------+-------------+-------------+-------------+-------------+-------------+
|   test |   batch = 1 |   batch = 2 |   batch = 3 |   batch = 4 |   batch = 5 |
+========+=============+=============+=============+=============+=============+
|      1 |          10 |           8 |           8 |           8 |           7 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      2 |           9 |           8 |           8 |           8 |           7 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      3 |           9 |           8 |           8 |           8 |           7 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      4 |           9 |           8 |           8 |           8 |           7 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      5 |          10 |           8 |           8 |           8 |           7 |
+--------+-------------+----

Плохо подобранные константы в более сложных методах могут ухудшить сходимость

In [23]:
visualize_batch_testing(5, lambda batch: exponential_learning_scheduler(0.9, 0.2, batch, 5))

+--------+-------------+-------------+-------------+-------------+-------------+
|   test |   batch = 1 |   batch = 2 |   batch = 3 |   batch = 4 |   batch = 5 |
+========+=============+=============+=============+=============+=============+
|      1 |          20 |          15 |          14 |          13 |          14 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      2 |          18 |          15 |          14 |          14 |          14 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      3 |          19 |          15 |          14 |          14 |          14 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      4 |          18 |          15 |          14 |          13 |          13 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      5 |          19 |          15 |          14 |          14 |          14 |
+--------+-------------+----

In [24]:
visualize_batch_testing(5, lambda batch: exponential_learning_scheduler(10, 0.01, batch, 5))

+--------+-------------+-------------+-------------+-------------+-------------+
|   test |   batch = 1 |   batch = 2 |   batch = 3 |   batch = 4 |   batch = 5 |
+========+=============+=============+=============+=============+=============+
|      1 |           9 |           8 |           8 |           8 |           7 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      2 |          10 |           8 |           8 |           8 |           7 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      3 |           9 |           8 |           8 |           8 |           7 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      4 |           9 |           8 |           8 |           7 |           7 |
+--------+-------------+-------------+-------------+-------------+-------------+
|      5 |          10 |           8 |           8 |           8 |           7 |
+--------+-------------+----